# FedEM Attack Analysis

TJ Kim
11.5.21

#### Summary:
- Load the locally trained model and FedEM model
- nodes = 3, mixtures = 3

In [1]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


### Import Relevant Libraries
Take it from the run_experiment.py folder

In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from torch.utils.tensorboard import SummaryWriter
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *

### Generate Aggregator Pre-requisite
- Clients, Test Clients, Ensemble_Learner
- Follow through the code in run_experiment.py

In [3]:
# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar10"
args_.method = "FedEM"
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= 3
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/21_12_02_first_transfers_xadv_train/'
args_.validation = False

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_)

==> Clients initialization..
===> Building data iterators..


100%|██████████████████████████████████████████| 80/80 [00:00<00:00, 209.48it/s]


===> Initializing clients..


100%|███████████████████████████████████████████| 80/80 [00:30<00:00,  2.60it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 2.292 | Train Acc: 12.159% |Test Loss: 2.292 | Test Acc: 12.248% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


In [4]:
# Import weights for aggregator
aggregator.load_state(args_.save_path)

# This is where the models are stored -- one for each mixture --> learner.model for nn
hypotheses = aggregator.global_learners_ensemble.learners

# obtain the state dict for each of the weights 
weights_h = []

for h in hypotheses:
    weights_h += [h.model.state_dict()]

In [5]:
weights = np.load("weights/cifar/21_12_02_first_transfers_xadv_train/train_client_weights.npy")
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

#print(weights)

# Set model weights
model_weights = []
num_models = 7

for i in range(num_models):
    model_weights += [weights[i]]
    
    
# Generate the weights to test on as linear combinations of the model_weights
models_test = []

for (w0,w1,w2) in model_weights:
    # first make the model with empty weights
    new_model = copy.deepcopy(hypotheses[0].model)
    new_model.eval()
    new_weight_dict = copy.deepcopy(weights_h[0])
    for key in weights_h[0]:
        new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
    new_model.load_state_dict(new_weight_dict)
    models_test += [new_model]

### Generate Data 

In [6]:
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].val_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
data_y = torch.stack(data_y)

In [7]:
# Create dataloader from validation dataset that allows for diverse batch size
dataloader = Custom_Dataloader(data_x, data_y)

### Set Up Transfer Attack Scenario
- Set up order of which attacks will take place -- keep the same transferer, but simply swap out the weights of the adversary and flush out the existing data points analysis was done on
- Make dictionaries beforehand recording all of the metrics

In [8]:
# Set Up Dictionaries -- list holds the adversary idx
logs_adv = []

for i in range(len(model_weights)):
    adv_dict = {}
    adv_dict['orig_acc_transfers'] = None
    adv_dict['orig_similarities'] = None
    adv_dict['adv_acc_transfers'] = None
    adv_dict['adv_similarities'] = None
    adv_dict['orig_target_hit'] = None
    adv_dict['adv_target_hit'] = None
    adv_dict['metric_variance'] = None
    adv_dict['metric_alignment'] = None
    adv_dict['metric_ingrad'] = None
    logs_adv += [adv_dict]

In [9]:
# Make transferer and Assign model index
victim_idxs = [0,1,2,3,4,5,6]

for adv_idx in victim_idxs:
    print("id", adv_idx)
    # Perform Attack
    t1 = Transferer(models_list=models_test, dataloader=dataloader)
    
    # atk Params setup
    t1.atk_params.set_params(batch_size=500, eps=0.1, alpha=0.05, iteration = 30,
                   target = 9, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x))
    
    t1.atk_params = PGD_Params()
    t1.atk_params.set_params(batch_size=500, iteration = 30,
                   target = 9, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                   step_size = 0.05, step_norm = "inf", eps = 4.5, eps_norm = 2)

    
    t1.generate_victims(victim_idxs)
    t1.generate_advNN(adv_idx)
    t1.generate_xadv(atk_type = "pgd")
    t1.send_to_victims(victim_idxs)
    # t1.check_empirical_metrics(orig_flag = True)
    # t1.check_empirical_metrics()
    
    # Log Performance
    logs_adv[adv_idx]['orig_acc_transfers'] = t1.orig_acc_transfers
    logs_adv[adv_idx]['orig_similarities'] = t1.orig_similarities
    logs_adv[adv_idx]['adv_acc_transfers'] = t1.adv_acc_transfers
    logs_adv[adv_idx]['adv_similarities'] = t1.adv_similarities
    logs_adv[adv_idx]['orig_target_hit'] = t1.orig_target_hit
    logs_adv[adv_idx]['adv_target_hit'] = t1.adv_target_hit
    
    logs_adv[adv_idx]['metric_variance'] = t1.metric_variance
    logs_adv[adv_idx]['metric_alignment'] = t1.metric_alignment
    logs_adv[adv_idx]['metric_ingrad'] = t1.metric_ingrad

id 0
id 1
id 2
id 3
id 4
id 5
id 6


### Print and Record Results
- Organize the results into a matrix and print them into an excel sheet
- Following Categories (sim_benign, sim_adv, target_adv,grad_alignment)

In [10]:
metrics = ['orig_acc_transfers','orig_similarities','adv_similarities','adv_target_hit','metric_alignment']

orig_acc = np.zeros([len(victim_idxs),len(victim_idxs)]) 
sim_benign = np.zeros([len(victim_idxs),len(victim_idxs)]) 
sim_adv = np.zeros([len(victim_idxs),len(victim_idxs)]) 
target_adv = np.zeros([len(victim_idxs),len(victim_idxs)]) 
# grad_align = np.zeros([len(victim_idxs),len(victim_idxs)]) 

In [11]:
for adv_idx in victim_idxs:
    for victim in victim_idxs:
        orig_acc[adv_idx,victim] = logs_adv[adv_idx][metrics[0]][victim].data.tolist()
        sim_benign[adv_idx,victim] = logs_adv[adv_idx][metrics[1]][victim].data.tolist()
        sim_adv[adv_idx,victim] = logs_adv[adv_idx][metrics[2]][victim].data.tolist()
        target_adv[adv_idx,victim] = logs_adv[adv_idx][metrics[3]][victim].data.tolist()
#         grad_align[adv_idx,victim] = logs_adv[adv_idx][metrics[4]][victim].data.tolist()

In [12]:
# Save to Excel file

## convert your array into a dataframe
orig_acc_df = pd.DataFrame(orig_acc)
sim_benign_df = pd.DataFrame(sim_benign)
sim_adv_df = pd.DataFrame(sim_adv)
target_adv_df = pd.DataFrame(target_adv)
# grad_align_df = pd.DataFrame(grad_align)

## save to xlsx file

#filepath = 'my_excel_file.xlsx'

# df.to_excel(filepath, index=False)

In [13]:
orig_acc_df

,0,1,2,3,4,5,6
0,0.796,0.794,0.786,0.704,0.722,0.794,0.800
1,0.788,0.778,0.784,0.686,0.700,0.790,0.786
2,0.766,0.752,0.744,0.710,0.660,0.766,0.768
3,0.774,0.764,0.768,0.698,0.728,0.768,0.774
4,0.782,0.772,0.774,0.704,0.718,0.782,0.776
5,0.802,0.794,0.788,0.752,0.746,0.810,0.802
6,0.800,0.768,0.756,0.710,0.702,0.804,0.794


In [14]:
sim_benign_df

,0,1,2,3,4,5,6
0,1.000,0.944,0.914,0.640,0.692,0.980,0.988
1,0.954,1.000,0.972,0.620,0.704,0.948,0.956
2,0.904,0.966,1.000,0.636,0.660,0.898,0.912
3,0.612,0.626,0.642,1.000,0.852,0.606,0.612
4,0.672,0.710,0.732,0.866,1.000,0.670,0.676
5,0.972,0.916,0.904,0.660,0.686,1.000,0.968
6,0.986,0.946,0.918,0.658,0.710,0.974,1.000


In [15]:
sim_adv_df

,0,1,2,3,4,5,6
0,1.000,0.914,0.850,0.342,0.410,0.976,0.996
1,0.950,1.000,0.970,0.406,0.498,0.926,0.952
2,0.936,0.976,1.000,0.480,0.536,0.912,0.932
3,0.410,0.464,0.478,1.000,0.752,0.402,0.410
4,0.466,0.502,0.534,0.778,1.000,0.450,0.464
5,0.970,0.864,0.830,0.356,0.412,1.000,0.960
6,0.990,0.910,0.856,0.396,0.446,0.980,1.000


In [16]:
target_adv_df

,0,1,2,3,4,5,6
0,0.588,0.524,0.476,0.104,0.112,0.598,0.584
1,0.512,0.526,0.506,0.094,0.124,0.500,0.514
2,0.506,0.532,0.530,0.148,0.168,0.492,0.504
3,0.148,0.168,0.176,0.408,0.238,0.144,0.148
4,0.188,0.212,0.226,0.274,0.374,0.174,0.190
5,0.594,0.506,0.476,0.110,0.118,0.622,0.590
6,0.586,0.522,0.470,0.126,0.126,0.586,0.578


In [17]:
# grad_align_df

In [18]:
# Average all the information together and present
orig_acc_mean = np.mean(orig_acc)
sim_benign_mean = np.mean((sim_benign.sum(1)-1)/(sim_benign.shape[1]-1))
sim_adv_mean = np.mean((sim_adv_df.sum(1)-1)/(sim_adv_df.shape[1]-1))
target_adv_mean = np.mean((target_adv.sum(1)-1)/(target_adv.shape[1]-1))
# grad_align_mean = np.mean((grad_align.sum(1)-0)/(grad_align.shape[1]-0))

print("orig_acc_mean", orig_acc_mean)
print("Sim Benign Mean", sim_benign_mean)
print("Sim ADV Mean", sim_adv_mean)
print("Target ADV Mean", target_adv_mean)
# print("Grad Align Mean", grad_align_mean)

orig_acc_mean 0.7608979952578642
Sim Benign Mean 0.8062381389595213
Sim ADV Mean 0.69147622301465
Target ADV Mean 0.25523811472313745


In [19]:
# Calculate average distance norm across examples
delta = t1.x_adv - t1.x_orig
dnorm = delta.view(delta.shape[0], -1).norm(2, dim=1)
print(torch.mean(dnorm))
print(torch.max(dnorm))

tensor(4.4966, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(4.5000, device='cuda:0', grad_fn=<MaxBackward1>)


In [20]:
# IFSGM
4.5347 # mean
4.8902 # Max
0.8203 # Mean transfer rate

0.8203

In [21]:
# PGD
4.4978 # mean perturbation
4.5000 # Max perturbation
0.8594 # Mean transfer rate

0.8594